In [54]:
#import all packages required
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
from sklearn.preprocessing import scale
from sklearn.cluster import DBSCAN

print('library imported successfully')

library imported successfully


In [2]:
with open('C:/Users/Jethr/OneDrive/IBM Certificates/Machine Learning/newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

New York has 5 boroughs and 306 neighborhoods.


In [4]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="lol2")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Manhattan has {} boroughs and {} neighborhoods.'.format(
        len(manhattan_data['Borough'].unique()),
        manhattan_data.shape[0]
    )
)

Manhattan has 1 boroughs and 40 neighborhoods.


In [6]:
print('The coordinates of Manhattan are {}, {}.'.format(latitude, longitude))
manhattan_data.head()

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The coordinates of Manhattan are 40.7900869, -73.9598295.


In [10]:
CLIENT_ID = '4XBXSKO2M0MAPRWO3LTW55IK0RHJ20XINY4BYNSAMSZG3STF' 
CLIENT_SECRET = 'NSMD1BCKJ2V3AF2PMUPOLJZMOXZ3VN0HCZNQEVAJLG2WYHOS'
VERSION = '20180605' # Foursquare API version
limit = 50
rad = 250

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [25]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
2,Marble Hill,40.876551,-73.91066,Subway Sandwiches,40.874667,-73.909586,Sandwich Place
3,Marble Hill,40.876551,-73.91066,CTown Supermarkets,40.876218,-73.908541,Supermarket
4,Marble Hill,40.876551,-73.91066,Metro North - Marble Hill Train Station,40.874698,-73.912052,Train Station


In [14]:
manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
manhattan_onehot.columns.values.tolist()

['Neighborhood',
 'Accessories Store',
 'Adult Boutique',
 'Afghan Restaurant',
 'American Restaurant',
 'Antique Shop',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Austrian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Basketball Court',
 'Beer Bar',
 'Beer Garden',
 'Beer Store',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Breakfast Spot',
 'Bridal Shop',
 'Bubble Tea Shop',
 'Building',
 'Burger Joint',
 'Burrito Place',
 'Bus Station',
 'Bus Stop',
 'Café',
 'Candy Store',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Circus',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Academic Building',
 'College Bookstore',
 'Comedy Club',
 'Community Center',
 'Concert Hall',
 'Convenience Store',
 'Cosmetics Shop',
 'Cuban Restau

In [17]:
manhattan_onehot_selected=manhattan_onehot[['Neighborhood','Café','Coffee Shop','Cantonese Restaurant','Pet Café','Convenience Store']]
manhattan_onehot_selected.head()

,Neighborhood,Café,Coffee Shop,Cantonese Restaurant,Pet Café,Convenience Store
0,Marble Hill,0,0,0,0,0
1,Marble Hill,0,0,0,0,0
2,Marble Hill,0,0,0,0,0
3,Marble Hill,0,0,0,0,0
4,Marble Hill,0,0,0,0,0


In [19]:
manhattan_grouped = manhattan_onehot_selected.groupby('Neighborhood').sum().reset_index()
manhattan_grouped

,Neighborhood,Café,Coffee Shop,Cantonese Restaurant,Pet Café,Convenience Store
0,Battery Park City,0,1,0,0,0
1,Carnegie Hill,3,1,0,0,0
2,Central Harlem,1,0,0,0,0
3,Chelsea,2,1,0,0,0
4,Chinatown,1,0,1,0,0
5,Civic Center,0,2,0,0,0
6,Clinton,1,1,0,0,0
7,East Harlem,0,0,0,0,0
8,East Village,0,2,0,1,0
9,Financial District,1,6,0,0,0


In [23]:
num_top_venues = 5
for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','Qty']
    temp = temp.iloc[1:]
    temp['Qty'] = temp['Qty'].astype(float)
    temp = temp.round({'Qty': 2})
    print(temp.sort_values('Qty', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                  venue  Qty
0           Coffee Shop  1.0
1                  Café  0.0
2  Cantonese Restaurant  0.0
3              Pet Café  0.0
4     Convenience Store  0.0


----Carnegie Hill----
                  venue  Qty
0                  Café  3.0
1           Coffee Shop  1.0
2  Cantonese Restaurant  0.0
3              Pet Café  0.0
4     Convenience Store  0.0


----Central Harlem----
                  venue  Qty
0                  Café  1.0
1           Coffee Shop  0.0
2  Cantonese Restaurant  0.0
3              Pet Café  0.0
4     Convenience Store  0.0


----Chelsea----
                  venue  Qty
0                  Café  2.0
1           Coffee Shop  1.0
2  Cantonese Restaurant  0.0
3              Pet Café  0.0
4     Convenience Store  0.0


----Chinatown----
                  venue  Qty
0                  Café  1.0
1  Cantonese Restaurant  1.0
2           Coffee Shop  0.0
3              Pet Café  0.0
4     Convenience Store  0.0


----Civic Cente

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.02,0.00,0.00,0.02,0.000000
1,Carnegie Hill,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.02,0.04,0.00,0.00,0.040000
2,Central Harlem,0.00,0.00,0.00,0.068182,0.045455,0.00,0.00,0.00,0.022727,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000
3,Chelsea,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.00,0.02,0.000000
4,Chinatown,0.00,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,...,0.04,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000
5,Civic Center,0.00,0.00,0.00,0.000000,0.040000,0.02,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.00,0.00,0.020000
6,Clinton,0.00,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.02,0.04,0.00,0.00,0.000000
7,East Harlem,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000
8,East Village,0.00,0.00,0.00,0.000000,0.040000,0.02,0.00,0.00,0.000000,...,0.04,0.00,0.00,0.000000,0.00,0.06,0.04,0.00,0.00,0.000000
9,Financial District,0.02,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.04,0.00,0.00,0.000000


In [42]:
manhattan_grouped.shape

(40, 274)

----Battery Park City----
              venue  freq
0       Coffee Shop  0.10
1              Park  0.08
2  Department Store  0.04
3         BBQ Joint  0.04
4        Food Court  0.04


----Carnegie Hill----
               venue  freq
0               Café  0.08
1        Pizza Place  0.06
2                Spa  0.06
3          Bookstore  0.06
4  French Restaurant  0.04


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.05
2  American Restaurant  0.05
3   Seafood Restaurant  0.05
4    French Restaurant  0.05


----Chelsea----
                venue  freq
0      Ice Cream Shop  0.08
1           Nightclub  0.06
2               Hotel  0.06
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.12
1  Vietnamese Restaurant  0.04
2       Asian Restaurant  0.04
3         Sandwich Place  0.04
4                    Bar  0.04


----Civic Center----
                  

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Battery Park City,Coffee Shop,Convenience Store,Pet Café
1,Carnegie Hill,Café,Coffee Shop,Convenience Store
2,Central Harlem,Café,Convenience Store,Pet Café
3,Chelsea,Café,Coffee Shop,Convenience Store
4,Chinatown,Cantonese Restaurant,Café,Convenience Store


In [42]:
# set number of clusters
kclusters = 3

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 2, 1, 0, 1, 1, 0, 0])

In [43]:
manhattan_grouped_clustering.head()

,Café,Coffee Shop,Cantonese Restaurant,Pet Café,Convenience Store
0,0,1,0,0,0
1,3,1,0,0,0
2,1,0,0,0,0
3,2,1,0,0,0
4,1,0,1,0,0


In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Convenience Store,Pet Café,Cantonese Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,1,Cantonese Restaurant,Café,Convenience Store
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Coffee Shop,Convenience Store
3,Manhattan,Inwood,40.867684,-73.921210,2,Café,Coffee Shop,Convenience Store
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Café,Coffee Shop,Convenience Store


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
2,Marble Hill,40.876551,-73.91066,Subway Sandwiches,40.874667,-73.909586,Sandwich Place
3,Marble Hill,40.876551,-73.91066,CTown Supermarkets,40.876218,-73.908541,Supermarket
4,Marble Hill,40.876551,-73.91066,Metro North - Marble Hill Train Station,40.874698,-73.912052,Train Station


In [187]:
coffee_list=['Neighborhood','Café','Coffee Shop','Cantonese Restaurant','Pet Café','Convenience Store']
coffee_loc = manhattan_venues[manhattan_venues['Venue Category'].isin(coffee_list)]
coffee_loc=coffee_loc.reset_index(drop=True)

In [188]:
#coffee_loc_cluser = scale(coffee_loc[['Venue Latitude', 'Venue Longitude']])

In [189]:
coffee_loc_cluser = scale(coffee_loc[['Neighborhood Longitude', 'Neighborhood Latitude','Venue Latitude', 'Venue Longitude']])

In [191]:
#clustering =  KMeans(n_clusters=4, random_state=12).fit(coffee_loc_cluser)
# check cluster labels generated for each row in the dataframe

clustering=DBSCAN(eps=0.5, min_samples=2).fit(coffee_loc_cluser)
clustering.labels_

array([ 0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  0,
        0,  0,  0,  0,  0,  0, -1,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0], dtype=int64)

In [192]:
ncl = len(list(set(clustering.labels_)))
ncl

6

In [193]:
# add clustering labels
coffee_loc2 = coffee_loc.iloc[:,:]
coffee_loc2.insert(0, 'Cluster Labels', clustering.labels_)
coffee_loc2.head()

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Chinatown,40.715618,-73.994279,Kings Kitchen 金煌煲煲好,40.713852,-73.993640,Cantonese Restaurant
1,0,Chinatown,40.715618,-73.994279,S.Wan Cafe 洋紫荊咖啡室,40.717269,-73.992412,Café
2,1,Washington Heights,40.851903,-73.936900,Green Juice Cafe,40.851898,-73.934827,Café
3,1,Washington Heights,40.851903,-73.936900,Starbucks,40.850961,-73.938330,Coffee Shop
4,1,Washington Heights,40.851903,-73.936900,Filtered Coffee,40.851607,-73.935014,Café


In [194]:
# create map
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ncl)
ys = [i + x + (i*x)**2 for i in range(ncl)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(coffee_loc2['Venue Latitude'], coffee_loc2['Venue Longitude'], coffee_loc2['Venue'], coffee_loc2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

In [196]:
final_group=coffee_loc2.loc[coffee_loc2['Cluster Labels'] == 0, coffee_loc2.columns[[1] + list(range(5, coffee_loc2.shape[1]))]]
final_group=final_group.reset_index(drop = True)
final_group.head()

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.713852,-73.993640,Cantonese Restaurant
1,Chinatown,40.717269,-73.992412,Café
2,Upper East Side,40.773697,-73.960862,Café
3,Upper East Side,40.773535,-73.959670,Coffee Shop
4,Upper East Side,40.773543,-73.959836,Coffee Shop


In [197]:
final_lat = final_group['Venue Latitude'].mean()
final_log = final_group['Venue Longitude'].mean()

print(final_lat)
print(final_log)

40.73965063567979
-73.98813181909837


In [200]:
label = folium.Popup('Projected Warehouse')
folium.CircleMarker(
        [final_lat, final_log],
        radius=6,
        popup=label,
        color=rainbow[2],
        fill=True,
        fill_color=rainbow[2],
        fill_opacity=0.8).add_to(map_clusters2)
map_clusters2